In [1]:
import os

import pandas as pd

movie_dataset_path = os.path.join(os.getcwd(), 'movie-dataset', 'movies.csv')
rating_dataset_path = os.path.join(os.getcwd(), 'movie-dataset', 'ratings.csv')

movie_dataset = pd.read_csv(movie_dataset_path, sep='::', engine='python')
rating_dataset = pd.read_csv(rating_dataset_path, sep='::', engine='python')

movie_dataset = movie_dataset.dropna()
rating_dataset = rating_dataset.dropna()